# Model Analysis for VetLLM

Explore the fine-tuned model's performance and generate predictions interactively.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from pathlib import Path
import matplotlib.pyplot as plt

# Load model and tokenizer
model_path = '../models/vetllm-finetuned'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model.eval()

# Example clinical notes
notes = [
    "Dog, 4 years old, Golden Retriever. Presents with lethargy and loss of appetite.",
    "Cat, 2 years old, Siamese. Shows signs of seizures and twitching.",
    "Horse, 8 years old, Thoroughbred. Lame on left hind leg, joint swelling."
]

def generate_prediction(note, max_new_tokens=100):
    prompt = (f"Below is an instruction that describes a veterinary diagnosis task, paired with a clinical note. "
              f"Write a response that predicts the SNOMED-CT diagnosis codes.\n\n"
              f"### Instruction:\nAnalyze the veterinary clinical note and predict SNOMED-CT diagnosis codes.\n\n"
              f"### Input:\nClinical Note: {note}\n\n### Response:\n")
    inputs = tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# Generate predictions
for i, note in enumerate(notes):
    print(f"---- Clinical Note {i+1} ----")
    print(note)
    pred = generate_prediction(note)
    print(f"Prediction: {pred}\n")